In [81]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import EasyEnsemble
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
directory='/content/drive/My Drive/x_financial'
os.chdir(directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv('rmnan.csv')
test = pd.read_csv('test.csv')
train = train.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
train = train.dropna(axis=1,how='any')
col = train.columns.to_list()
train = train[col]
test = test[col]
train_y = train['bad30']
train = train.drop(columns=['bad30'])
test_y = test['bad30']
test = test.drop(columns=['bad30'])

In [0]:
train = pd.read_csv('rmnan.csv')
test = pd.read_csv('test.csv')
col = [ 'AP003', 'TD001', 'MB005', 'TD013', 'TD009', 'CR008',
       'TD005', 'TD014', 'CR021', 'TD010', 'CR015', 'CR009', 'CR018', 
       'CD121', 'ym0276', 'ym0358', 'CD117', 'ym0316', 'ym0305',
       'ym2333', 'CR017', 'MB001', 'ym0322', 'ym0090', 'CD123', 'TD006',
       'CD016', 'CD109', 'ym0265', 'ym0306', 'CD002', 'CD178',
       'ym0366', 'CD169', 'CD122', 'ym0004', 'ym0009', 'CD120', 'ym2037',
       'CD003', 'ym2055', 'ym0079', 'CD011', 'ym0299', 'ym2041', 'CD172',
       'ym0304', 'AP007', 'ym0270', 'CD008', 'ym0361', 'CD110', 'ym0038',
       'ym2156', 'ym0327', 'ym0290', 'AP002', 'ym0362', 'ym0339', 'ym0295','bad30']
train = train[col]
test = test[col]
train = train.dropna(axis=0,how='any')
train_y = train['bad30']
train = train.drop(columns=['bad30'])
test_y = test['bad30']
test = test.drop(columns=['bad30'])

col = [ 'AP003', 'TD001', 'TD016', 'MB005', 'TD013', 'TD009', 'CR008',
       'TD005', 'CR020', 'TD014', 'CR021', 'TD010', 'CR015', 'CR009', 'CR018',
       'ym0383', 'CD121', 'ym0276', 'ym0358', 'CD117', 'ym0316', 'ym0305',
       'ym2333', 'CR017', 'MB001', 'ym0322', 'ym0090', 'CD123', 'TD006',
       'CD016', 'CD109', 'ym0265', 'ym0306', 'CD002', 'CD178', 'ym0360',
       'ym0366', 'CD169', 'CD122', 'ym0004', 'ym0009', 'CD120', 'ym2037',
       'CD003', 'ym2055', 'ym0079', 'CD011', 'ym0299', 'ym2041', 'CD172',
       'ym0304', 'AP007', 'ym0270', 'CD008', 'ym0361', 'CD110', 'ym0038',
       'ym2156', 'ym0327', 'ym0290', 'AP002', 'ym0362', 'ym0339', 'ym0295']

In [83]:
ee = EasyEnsemble(n_subsets=4, random_state=42, replacement = False)
X_res, y_res = ee.fit_resample(train, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class EasyEnsemble is deprecated; EasyEnsemble is deprecated in 0.4 and will be removed in 0.6. Use EasyEnsembleClassifier instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
p = []
p_y = []
for i in range(len(X_res)):
  classifier = AdaBoostClassifier(base_estimator=None, n_estimators=1000, learning_rate=0.05, random_state=42)
  classifier.fit(X_res[i],y_res[i])
  pred = classifier.predict_proba(X_res)
  pred_y = classifier.predict_proba(test)
  p.append(pred)
  p_y.append(pred_y)

In [0]:
def return_prob(p):
  prob = []
  for j in range(len(p)):
    element = []
    for i in range(len(p[j])):
      element.append(p[j][i][1])
    prob.append(element)
  return prob

In [0]:
prob = return_prob(p)
prob_y = return_prob(p_y)

In [0]:
def choose_threshold(prob,test):
  accuracy = []
  r = []
  # threshold = [0.50,0.501,0.502,0.503,0.504,0.505,0.506,0.507,0.508]
  threshold = list(np.arange(0.5,0.503,0.0001))
  for i in range(len(threshold)):
    result = []
    for j in range(len(prob)):
      if prob[j] >= threshold[i]:
        result.append(1)
      else:
        result.append(0)
    acc = sum(result == test)/len(test)
    accuracy.append(acc)
    ra = sum(result)/len(result)
    r.append(ra)
  dictionary = dict(zip(threshold, accuracy))
  ratio = dict(zip(threshold, r))
  return dictionary, ratio

In [0]:
choose_threshold(prob[0],train_y) #0.502

In [0]:
choose_threshold(prob[1]) #0.5022

In [0]:
choose_threshold(prob[2]) #0.5022

In [0]:
choose_threshold(prob[3])

In [112]:
"""
accuracy in training set
"""
prediction = []
pred_total = []
for i in range(len(prob[0])):
  p = prob[0][i] + prob[1][i] + prob[2][i] + prob[3][i] #+ prob[4][i] + prob[5][i] + prob[6][i] + prob[7][i] +prob[8][i] + prob[9][i]
  pred_total.append(p)
for j in range(len(pred_total)):
  if pred_total[j] >= 0.502*4:
    prediction.append(1)
  else:
    prediction.append(0)
print(sum(prediction)/len(prediction))
print(sum(prediction == train_y)/len(prediction))

0.1851158144471499
0.7472217019416869


In [117]:
"""
accuracy in test set
"""
prediction = []
pred_total = []
for i in range(len(prob_y[0])):
  p = prob_y[0][i] + prob_y[1][i] + prob_y[2][i] + prob_y[3][i] #+ prob[4][i] + prob[5][i] + prob[6][i] + prob[7][i] +prob[8][i] + prob[9][i]
  pred_total.append(p)
for j in range(len(pred_total)):
  if pred_total[j] >= 0.502*4:
    prediction.append(1)
  else:
    prediction.append(0)
print(sum(prediction)/len(prediction))
print(sum(prediction == test_y)/len(prediction))

0.19821848571285594
0.7312715808437171


In [0]:
def ks_statistic(Y,Y_hat):
	data = {"Y":Y,"Y_hat":Y_hat}
	df = pd.DataFrame(data)
	bins = np.array([-0.1,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
	category = pd.cut(df["Y_hat"],bins=bins)
	category = category.sort_values()
	#max_index = len(np.unique(df["Y_hat"]))
	Y = df.ix[category.index,:]['Y']
	Y_hat = df.ix[category.index,:]['Y_hat']
	df2 = pd.concat([Y,Y_hat],axis=1)
	df3 = pd.pivot_table(df2,values = ['Y_hat'],index ='Y_hat',columns='Y',aggfunc=len,fill_value=0)
	df4 = np.cumsum(df3)
	df5 = df4/df4.iloc[:,1].max()
	ks = max(abs(df5.iloc[:,0] - df5.iloc[:,1]))
	return ks/len(bins)

In [0]:
def ks_calc_cross(data,pred,y_label):
	crossfreq = pd.crosstab(data[pred[0]],data[y_label[0]])
	crossdens = crossfreq.cumsum(axis=0) / crossfreq.sum()
	crossdens['gap'] = abs(crossdens[0] - crossdens[1])
	ks = crossdens[crossdens['gap'] == crossdens['gap'].max()]
	return ks,crossdens

In [0]:
def array_trans(x, y): 
    # x: prediction
    # y: actual value
    a = np.array(x) 
    b = np.array(y)
    result = np.vstack((a,b))
    
    return result  

def cal_rate(result, thres):
    all_number = len(result[0])
    # print all_number
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for item in range(all_number):
        prediction = result[0][item]
        if prediction >= thres:
            prediction = 1
        if prediction == 1:
            if result[1][item] == 1:
                TP += 1
            else:
                FP += 1
        else:
            if result[1][item] == 0:
                TN += 1
            else:
                FN += 1
    # print TP+FP+TN+FN
    accracy = float(TP+TN) / float(all_number)
    if TP+FP == 0:
        precision = 0
    else:
        precision = float(TP) / float(TP+FP)
    TPR = float(TP) / float(TP+FN)
    TNR = float(TN) / float(FP+TN)
    FNR = float(FN) / float(TP+FN)
    FPR = float(FP) / float(FP+TN)
    print('accuracy')
    print(accracy)
    print('precision')
    print(precision)
    print('sensitivity')
    print(TPR)
    print('specificity')
    print(TNR)
    # print accracy, precision, TPR, TNR, FNR, FPR
    return accracy, precision, TPR, TNR, FNR, FPR

def AUROC_func(result):
    prob = np.hstack((np.random.rand(98),[0,1]))
    threshold_vaule = sorted(prob)
    threshold_num = len(threshold_vaule)
    #get a array for plot  
    accracy_array = np.zeros(threshold_num)
    precision_array = np.zeros(threshold_num)
    TPR_array = np.zeros(threshold_num)
    TNR_array = np.zeros(threshold_num)
    FNR_array = np.zeros(threshold_num)
    FPR_array = np.zeros(threshold_num)

    #calculate all the rates
    for thres in range(threshold_num):
        accracy_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[0]
        precision_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[1]
        TPR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[2]
        TNR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[3]
        FNR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[4]
        FPR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[5]
        print('threshold #: %d, threshold: %f' % (thres+1,threshold_vaule[thres]))

    #draw the plot with the rates array

    AUC = np.trapz(-TPR_array, FPR_array)
    threshold = np.argmin(abs(FNR_array - FPR_array))
    EER = (FNR_array[threshold]+FPR_array[threshold])/2
    plt.plot(FPR_array, TPR_array)
    plt.title('roc')
    plt.xlabel('FPR_array')
    plt.ylabel('TPR_array')
    plt.show()
    
    return AUC

In [0]:
def createGains(Y,Yhat):
    data = {"Y":test_y,'Y_hat':Yhat}
    test_scores = pd.DataFrame(data)
    test_scores.columns = ['true', 'predict']

    #sort on prediction (descending), add id, and decile for groups containing 1/10 of datapoints
    test_scores = test_scores.sort_values(by='predict',ascending=False)
    test_scores['row_id'] = range(0,0+len(test_scores))
    test_scores['decile'] = ( test_scores['row_id'] / (len(test_scores)/10) ).astype(int)
    #see count by decile
    test_scores.loc[test_scores['decile'] == 10]=9
    test_scores['decile'].value_counts()

    #create gains table
    gains = test_scores.groupby('decile')['true'].agg(['count','sum'])
    gains.columns = ['count','actual']
    gains

    #add features to gains table
    gains['non_actual'] = gains['count'] - gains['actual']
    gains['cum_count'] = gains['count'].cumsum()
    gains['cum_actual'] = gains['actual'].cumsum()
    gains['cum_non_actual'] = gains['non_actual'].cumsum()
    gains['percent_cum_actual'] = (gains['cum_actual'] / np.max(gains['cum_actual'])).round(2)
    gains['percent_cum_non_actual'] = (gains['cum_non_actual'] / np.max(gains['cum_non_actual'])).round(2)
    gains['if_random'] = np.max(gains['cum_actual']) /10 
    gains['if_random'] = gains['if_random'].cumsum()
    gains['lift'] = (gains['cum_actual'] / gains['if_random']).round(2)
    gains['K_S'] = np.abs( gains['percent_cum_actual'] -  gains['percent_cum_non_actual'] ) * 100
    gains['gain']=(gains['cum_actual']/gains['cum_count']*100).round(2)
    gains = pd.DataFrame(gains)
    return(gains)

In [118]:
ks_statistic(test_y,prediction)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


0.29436435124508525

In [122]:
createGains(test_y,prediction)

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,610.0,1389.0,1999,610.0,1389.0,0.16,0.09,381.5,1.60,7.0,30.52
1,1998,600.0,1398.0,3997,1210.0,2787.0,0.32,0.17,763.0,1.59,15.0,30.27
2,1998,331.0,1667.0,5995,1541.0,4454.0,0.40,0.28,1144.5,1.35,12.0,25.70
3,1999,322.0,1677.0,7994,1863.0,6131.0,0.49,0.38,1526.0,1.22,11.0,23.30
4,1998,306.0,1692.0,9992,2169.0,7823.0,0.57,0.48,1907.5,1.14,9.0,21.71
5,1998,365.0,1633.0,11990,2534.0,9456.0,0.66,0.58,2289.0,1.11,8.0,21.13
6,1999,353.0,1646.0,13989,2887.0,11102.0,0.76,0.69,2670.5,1.08,7.0,20.64
7,1998,311.0,1687.0,15987,3198.0,12789.0,0.84,0.79,3052.0,1.05,5.0,20.00
8,1998,291.0,1707.0,17985,3489.0,14496.0,0.91,0.90,3433.5,1.02,1.0,19.40


In [123]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y,prediction)

0.5723751688528641